Merge targets (last phdos peak), robocrys and lobsterpy descriptions (smallest basis) for matbench dataset.

In [1]:
from collections import Counter
import pandas as pd

In [2]:
targets = pd.read_csv("Targets/ph_peak.csv")
targets.set_index("mpid", inplace=True, drop=True)

In [3]:
robocrys = pd.read_json("Robocrys/df_robocrys_info.json")

In [4]:
lobsterpy = pd.read_csv("LobsterPy/LobsterPy_cba_text_small.csv")
lobsterpy.set_index("Unnamed: 0", inplace=True, drop=True)

In [5]:
df1 = targets.merge(lobsterpy, left_index=True, right_index=True)
overall = df1.merge(robocrys, left_index=True, right_index=True)

In [6]:
# Drop unnecessary columns
overall.drop(columns=['structure', 'robocrys_condensed_structure', 'cation_anion_bonds_cba', 'all_bonds_cba'], inplace=True)

In [7]:
for col in overall.columns:
    print(col, Counter([type(v) for v in overall[col].values]))
# -> go for all_bonds version

last phdos peak Counter({<class 'numpy.float64'>: 1264})
cation_anion_bonds_text Counter({<class 'str'>: 1259, <class 'float'>: 5})
all_bonds_text Counter({<class 'str'>: 1264})
robocrys_description_ca Counter({<class 'str'>: 1257, <class 'NoneType'>: 7})
robocrys_description_all Counter({<class 'str'>: 1257, <class 'NoneType'>: 7})


#### All bonds, both descriptions

In [8]:
# All bonds, both descriptions
all_bonds = overall.copy()
all_bonds["instruction"] = "What is the frequency of the highest frequency optical phonon mode peak in units of 1/cm given the following description?"

for mp_id, row in all_bonds.iterrows():
    if isinstance(row["robocrys_description_all"], str):
        all_bonds.at[mp_id, "input"] = row["all_bonds_text"] + " " + row["robocrys_description_all"]
    else:
        all_bonds.at[mp_id, "input"] = row["all_bonds_text"]
        print(f"No robocrys description for {mp_id}")
        
    all_bonds.at[mp_id, "output"] = f"The frequency of the highest frequency optical phonon mode peak is " + str(round(row["last phdos peak"], 4)) + " 1/cm."
    
all_bonds.drop(columns=["cation_anion_bonds_text", "robocrys_description_ca"], inplace=True)

No robocrys description for mp-1634
No robocrys description for mp-19795
No robocrys description for mp-23116
No robocrys description for mp-27716
No robocrys description for mp-602
No robocrys description for mp-7581
No robocrys description for mp-7988


In [9]:
all_bonds.drop(columns=['last phdos peak', 'all_bonds_text', 'robocrys_description_all'], inplace=True)
all_bonds.to_json("Data_unsloth/unsloth_input_last-phdos_all-bonds_robocrys_and_lobsterpy.json")

#### Only LobsterPy text descriptions

In [10]:
# Only LobsterPy text descriptions
all_bonds = overall.copy()
all_bonds["instruction"] = "What is the frequency of the highest frequency optical phonon mode peak in units of 1/cm given the following description?"

for mp_id, row in all_bonds.iterrows():

    all_bonds.at[mp_id, "input"] = row["all_bonds_text"]
        
    all_bonds.at[mp_id, "output"] = f"The frequency of the highest frequency optical phonon mode peak is " + str(round(row["last phdos peak"], 4)) + " 1/cm."
    
all_bonds.drop(columns=["cation_anion_bonds_text", "robocrys_description_ca"], inplace=True)

In [11]:
all_bonds.drop(columns=['last phdos peak', 'all_bonds_text', 'robocrys_description_all'], inplace=True)
all_bonds.to_json("Data_unsloth/unsloth_input_last-phdos_all-bonds_only_lobsterpy.json")

#### Only Robocrys text descriptions

In [12]:
# Only Robocrys text descriptions
all_bonds = overall.copy()
all_bonds["instruction"] = "What is the frequency of the highest frequency optical phonon mode peak in units of 1/cm given the following description?"

for mp_id, row in all_bonds.iterrows():

    if isinstance(row["robocrys_description_all"], str):
        all_bonds.at[mp_id, "input"] = row["robocrys_description_all"]
        
    all_bonds.at[mp_id, "output"] = f"The frequency of the highest frequency optical phonon mode peak is " + str(round(row["last phdos peak"], 4)) + " 1/cm."

missing_ids = [mp_id for mp_id, robotext in zip(all_bonds.index.values, all_bonds.robocrys_description_all.values) if robotext is None]
all_bonds.drop(missing_ids, inplace=True)
all_bonds.drop(columns=["cation_anion_bonds_text", "robocrys_description_ca"], inplace=True)

In [13]:
len(all_bonds)

1257

In [14]:
all_bonds.drop(columns=['last phdos peak', 'all_bonds_text', 'robocrys_description_all'], inplace=True)
all_bonds.to_json("Data_unsloth/unsloth_input_last-phdos_all-bonds_only_robocrys.json")